In [ ]:
#! /usr/bin/env python

import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn
import csv

# Parameters
# ==================================================

# Data Parameters
tf.flags.DEFINE_string("positive_data_file", "./data/rpolarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rpolarity.neg", "Data source for the negative data.")

# Eval Parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_string("checkpoint_dir", "", "Checkpoint directory from training run")
tf.flags.DEFINE_boolean("eval_train", False, "Evaluate on all training data")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")   #whether to log the placement of operations on devices


FLAGS = tf.flags.FLAGS  #define boolean command-line flags
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
    #Prints each flag's name (converted to uppercase) and its corresponding value
print("")

# CHANGE THIS: Load data. Load data here
if FLAGS.eval_train:
    #loads data from the specified positive and negative data files using the load_data_and_labels function
    x_raw, y_test = data_helpers.load(FLAGS.positive_data_file, FLAGS.negative_data_file)
    #converts the one-hot encoded labels (y_test) to class labels by taking the index of the maximum value along the specified axis (axis=1)
    y_test = np.argmax(y_test, axis=1)
else:
    # some default text data
    x_raw = ["a masterpiece four years in the making", "everything is off."]
    y_test = [1, 0]

# Map data into vocabulary
vocab_path = os.path.join(FLAGS.checkpoint_dir, "..", "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
#transform method of the vocabulary processor to map the test data (x_raw) into numerical representations based on the vocabulary
x_test = np.array(list(vocab_processor.transform(x_raw)))

print("\nEvaluating...\n")

# Evaluation
# loading the latest checkpoint file and creating a TensorFlow session with a specified configuration
# latest checkpoint file in the specified directory
checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)

#Creates a new TensorFlow graph
graph = tf.Graph()
with graph.as_default():
    #Configures a TensorFlow session (sess)
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,  # allows TensorFlow to place an operation on any available device if the specified device is unavailable
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    # sets sess as the default session
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate, Retrieves the tensor for predictions from the graph
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch, generate batches for one epoch of evaluation
        # batches are generated in order without shuffling
        batches = main.batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])
        #evaluates the model on the test data in batches,retrieves predictions from the specified tensor,final result is a NumPy array (all_predictions) containing the predictions for the entire test dataset
# Print accuracy if y_test is defined
if y_test is not None:
    #calculates the total number of correct predictions by summing the elements where predictions match ground truth labels.
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

# Save the evaluation to a csv
predictions_human_readable = np.column_stack((np.array(x_raw), all_predictions))
out_path = os.path.join(FLAGS.checkpoint_dir, "..", "prediction.csv")
print("Saving evaluation to {0}".format(out_path))
with open(out_path, 'w') as f:
    csv.writer(f).writerows(predictions_human_readable)